In [ ]:
import requests
import pprint
import pandas as pd
import geopandas as gpd

from covidcaremap.constants import *
from covidcaremap.data import (read_us_states_gdf,
                               read_us_counties_gdf,
                               external_data_path,
                               published_data_path)

In [ ]:
AIRTABLE_KEY = ''

In [ ]:
url = 'https://api.airtable.com/v0/appiZGFluszFy8q2r/County%20Overrides?maxRecords=100&view=Validated&sort%5B0%5D%5Bfield%5D=Source%20Date&sort%5B0%5D%5Bdirection%5D=desc'

headers = {'Authorization': f'Bearer {AIRTABLE_KEY}'}

r = requests.get(url, headers=headers)
records = r.json()['records']

In [ ]:
def calc_icu_missing(fields_dict):
    fields_dict_keys = list(fields_dict.keys())
    
    cols = ['Staffed ICU Beds',
            'ICU Beds Currently Available',
            'ICU Beds Currently Occupied',
            ]
    
    # at least 2 of the 3 above cols should be present
    assert len(set(fields_dict_keys) & set(cols)) >= 2
   
    if 'ICU Beds Currently Available' in fields_dict_keys and 'ICU Beds Currently Occupied' in fields_dict_keys:
        fields_dict['Staffed ICU Beds'] = fields_dict['ICU Beds Currently Available'] + fields_dict['ICU Beds Currently Occupied']        
    elif 'ICU Beds Currently Occupied' in fields_dict_keys:
        fields_dict['ICU Beds Currently Available'] = fields_dict['Staffed ICU Beds'] - fields_dict['ICU Beds Currently Occupied']
    else:
        fields_dict['ICU Beds Currently Occupied'] = fields_dict['Staffed ICU Beds'] - fields_dict['ICU Beds Currently Available']

    fields_dict['ICU Bed Occupancy Rate'] = fields_dict['ICU Beds Currently Occupied'] / fields_dict['Staffed ICU Beds']

    return fields_dict

In [ ]:
state_records = []
county_records = []

for rec in records:
    if 'County Name' in rec['fields'].keys(): county_records.append(rec)
    else: state_records.append(rec)

In [ ]:
county_records

In [ ]:
state_records

In [ ]:
state_records

In [ ]:
state_override_list = []
cols = ['State','ICU Bed Occupancy Rate','Staffed ICU Beds','Source Date','Manual Override New Data Source']
    
for rec in state_records:
    try: 
        icu_complete_rec = calc_icu_missing(rec['fields'])
        state_override_list.append({k: v for k, v in icu_complete_rec.items() if k in cols})
    except AssertionError: print(f'not enough ICU fields completed for record {rec["id"]}')
    

state_df = pd.DataFrame.from_dict(state_override_list)
state_df['Source Date'] = pd.to_datetime(state_df['Source Date'])

In [ ]:
county_override_list = []
cols = ['State','County Name','ICU Bed Occupancy Rate','Staffed ICU Beds','Source Date','Manual Override New Data Source']
    
for rec in county_records:
    try: 
        icu_complete_rec = calc_icu_missing(rec['fields'])
        county_override_list.append({k: v for k, v in icu_complete_rec.items() if k in cols})
    except AssertionError: print(f'not enough ICU fields completed for record {rec["id"]}')
    

county_df = pd.DataFrame.from_dict(county_override_list)
county_df['Source Date'] = pd.to_datetime(county_df['Source Date'])

In [ ]:
state_df_dedup = state_df.sort_values(['Source Date'],ascending=False).drop_duplicates('State')
county_df_dedup = county_df.sort_values(['Source Date'],ascending=False).drop_duplicates(['State', 'County Name'])

In [ ]:
state_df

In [ ]:
state_df_dedup

In [ ]:
county_df_dedup

In [ ]:
state_data = pd.read_csv(
    published_data_path('us_healthcare_capacity-state-CovidCareMap.csv'),
    dtype={'State': str }
)

county_data = pd.read_csv(
    published_data_path('us_healthcare_capacity-county-CovidCareMap.csv'),
    dtype={'State': str , 'fips_code': str}
)

In [ ]:
county_data.dtypes

In [ ]:
state_df_dedup = pd.merge(state_df_dedup, state_data, on='State', how='left', suffixes=('','_right'))


In [ ]:
state_df_dedup.drop(axis=1, 
                    columns=[col for col in state_df_dedup.columns if '_right' in col]
                            +['ICU Bed Source','ICU Bed Source Last Updated'], 
                    inplace=True)

In [ ]:
state_df_dedup

In [ ]:
county_df_dedup = pd.merge(county_df_dedup, county_data, on=['State','County Name'], how='left', suffixes=('','_right'))

In [ ]:
county_df_dedup.drop(axis=1, 
                     columns=[col for col in county_df_dedup.columns if '_right' in col]
                     +['ICU Bed Source','ICU Bed Source Last Updated'], 
                     inplace=True)

In [ ]:
county_df_dedup

In [ ]:
county_df_dedup['Manual Override Reason'] = 'Airtable update flow'
state_df_dedup['Manual Override Reason'] = 'Airtable update flow'

In [ ]:
state_df_dedup['Staffed ICU Beds [Per 1000 People]'] = round(state_df_dedup['Staffed ICU Beds'] / (state_df_dedup['Population'] / PER_CAPITA_BASE),3)
state_df_dedup['Staffed ICU Beds [Per 1000 Adults (20+)]'] = round(state_df_dedup['Staffed ICU Beds'] / (state_df_dedup['Population (20+)'] / PER_CAPITA_BASE),3)
state_df_dedup['Staffed ICU Beds [Per 1000 Elderly (65+)]'] = round(state_df_dedup['Staffed ICU Beds'] / (state_df_dedup['Population (65+)'] / PER_CAPITA_BASE),3)

In [ ]:
county_df_dedup['Staffed ICU Beds [Per 1000 People]'] = round(county_df_dedup['Staffed ICU Beds'] / (county_df_dedup['Population'] / PER_CAPITA_BASE),3)
county_df_dedup['Staffed ICU Beds [Per 1000 Adults (20+)]'] = round(county_df_dedup['Staffed ICU Beds'] / (county_df_dedup['Population (20+)'] / PER_CAPITA_BASE),3)
county_df_dedup['Staffed ICU Beds [Per 1000 Elderly (65+)]'] = round(county_df_dedup['Staffed ICU Beds'] / (county_df_dedup['Population (65+)'] / PER_CAPITA_BASE),3)

In [ ]:
county_df_dedup.T

In [ ]:
state_df_dedup.to_csv(external_data_path('covidcaremap-ushcsc-state-manual-override.csv'),index=False)
county_df_dedup.to_csv(external_data_path('covidcaremap-ushcsc-county-manual-override.csv'),index=False)